In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from mordred import Calculator, descriptors
from mordred.error import Missing
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier


In [2]:
chemtaste_df = pd.read_csv("ChemTastes_rdDescs_nonpeptides.csv")
chemtaste_df = chemtaste_df.iloc[:, 1:]
chemtaste_bittersweet_df = chemtaste_df[chemtaste_df['Class taste'].isin(['Bitterness', 'Sweetness'])]
chemtaste_bittersweet_df

,ID,Name,canonical SMILES,Class taste,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0001,(-)-Haematoxylin,Oc1cc2c(cc1O)C1c3ccc(c(c3OCC1(O)C2)O)O,Sweetness,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,...,0,0,0,0,0,0,0,0,0,0
1,0002,(+)-4β-hydroxyhernandulcin,CC(C)=CCCC(C)(O)C1CC(O)C(=CC1=O)C,Sweetness,11.931030,11.931030,0.069444,-1.054352,0.755242,27.444444,...,0,0,0,0,0,0,0,0,0,0
2,0003,(+)-Dihydroquercetin 3-acetate,CC(=O)OC1C(Oc2cc(cc(c2C1=O)O)O)c1ccc(c(c1)O)O,Sweetness,12.709217,12.709217,0.094012,-1.412606,0.477325,19.000000,...,0,0,0,0,0,0,0,0,0,0
3,0004,(+)-Dihydro-6-methoxy-luteolin 3-acetate,COc1c(cc2c(c1O)C(=O)C(OC(C)=O)C(O2)c1ccc(c(c1)...,Sweetness,12.921813,12.921813,0.155771,-1.473063,0.465280,18.370370,...,0,0,0,0,0,0,0,0,0,0
4,0005,(+)-Haematoxylin,Oc1cc2c(cc1O)C1c3ccc(c(c3OCC1(O)C2)O)O,Sweetness,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,2923,β-Colubrine,COc1ccc2c(c1)C13CCN4CC5=CCOC6CC(=O)N2C3C6C5CC14,Bitterness,13.298427,13.298427,0.040278,0.040278,0.716024,43.296296,...,0,0,0,0,0,0,0,0,0,0
2363,2924,β-D-Asp-L-Phe-OMe,COC(=O)C(Cc1ccccc1)NC(=O)CC(N)CC(O)=O,Bitterness,11.864139,11.864139,0.172040,-1.074904,0.585158,13.000000,...,0,0,0,0,0,0,0,0,0,0
2364,2925,β-D-Mannose,OCC1OC(O)C(O)C(O)C1O,Bitterness,9.119537,9.119537,0.525787,-1.568935,0.290153,49.250000,...,0,0,0,0,0,0,0,0,0,0
2365,2926,β-Glucogallin,OCC1OC(OC(=O)c2cc(c(c(c2)O)O)O)C(O)C(O)C1O,Bitterness,11.933569,11.933569,0.365963,-1.781427,0.236373,30.869565,...,0,0,0,0,0,0,0,0,0,0


In [3]:
chemtaste_bittersweet_df.head(30)

,ID,Name,canonical SMILES,Class taste,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0001,(-)-Haematoxylin,Oc1cc2c(cc1O)C1c3ccc(c(c3OCC1(O)C2)O)O,Sweetness,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,...,0,0,0,0,0,0,0,0,0,0
1,0002,(+)-4β-hydroxyhernandulcin,CC(C)=CCCC(C)(O)C1CC(O)C(=CC1=O)C,Sweetness,11.931030,11.931030,0.069444,-1.054352,0.755242,27.444444,...,0,0,0,0,0,0,0,0,0,0
2,0003,(+)-Dihydroquercetin 3-acetate,CC(=O)OC1C(Oc2cc(cc(c2C1=O)O)O)c1ccc(c(c1)O)O,Sweetness,12.709217,12.709217,0.094012,-1.412606,0.477325,19.000000,...,0,0,0,0,0,0,0,0,0,0
3,0004,(+)-Dihydro-6-methoxy-luteolin 3-acetate,COc1c(cc2c(c1O)C(=O)C(OC(C)=O)C(O2)c1ccc(c(c1)...,Sweetness,12.921813,12.921813,0.155771,-1.473063,0.465280,18.370370,...,0,0,0,0,0,0,0,0,0,0
4,0005,(+)-Haematoxylin,Oc1cc2c(cc1O)C1c3ccc(c(c3OCC1(O)C2)O)O,Sweetness,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,...,0,0,0,0,0,0,0,0,0,0
5,0006,(+)-Phyllodulcin 8-O-glucoside,COc1ccc(cc1O)C1Cc2cccc(c2C(=O)O1)OC1OC(CO)C(O)...,Sweetness,12.836097,12.836097,0.062831,-1.612695,0.395823,29.718750,...,0,0,0,0,0,0,0,0,0,0
6,0007,(+)-Quercitol,OC1CC(O)C(O)C(O)C1O,Sweetness,8.967037,8.967037,0.111713,-1.464120,0.265968,52.636364,...,0,0,0,0,0,0,0,0,0,0
7,0008*,(1-{4-[(hydroxyimino)methyl]cyclohex-3-en-1-yl...,CON=C(C)C1CCC(=CC1)C=NO,Sweetness,8.362702,8.362702,0.463697,0.463697,0.427107,23.714286,...,0,0,0,0,0,0,0,0,0,0
8,0009,"(2R,4R)-monatin (monosodium salt)",[Na+].NC(CC(O)(CC1=CNc2ccccc12)C(O)=O)C([O-])=O,Sweetness,11.312551,11.312551,0.000000,-2.273362,0.400702,14.818182,...,0,0,0,0,0,0,0,0,0,0
9,0010,"(2S,4aS,7R,8aS)-(-)-2-(3'-Hydroxy-4'-methoxyph...",COc1ccc(cc1O)C1OC2CC3CCC2(CS1)C3(C)C,Sweetness,10.006611,10.006611,0.030168,0.030168,0.878634,38.772727,...,1,0,0,0,0,0,0,0,0,0


In [4]:
missing_counts = chemtaste_bittersweet_df.isna().sum()
missing_proportion = missing_counts / len(chemtaste_bittersweet_df)

missing_df = pd.DataFrame({
    'missing_count': missing_counts,
    'missing_proportion': missing_proportion
})
missing_df.to_csv('output.csv', index=False)  # Excludes row indices
missing_df.head(20)

,missing_count,missing_proportion
ID,0,0.000000
Name,0,0.000000
canonical SMILES,0,0.000000
Class taste,0,0.000000
MaxAbsEStateIndex,0,0.000000
MaxEStateIndex,0,0.000000
MinAbsEStateIndex,0,0.000000
MinEStateIndex,0,0.000000
qed,0,0.000000
SPS,0,0.000000


In [72]:
# column_names = chemtaste_bittersweet_df.columns.tolist()  # Returns a list of column names
# print(column_names)

In [5]:
bittersweet_descriptors = chemtaste_bittersweet_df.iloc[:, 4:]

feature_selector = VarianceThreshold(threshold=0.01)
filter_bittersweet_descriptors = feature_selector.fit_transform(bittersweet_descriptors)
res_descriptors = bittersweet_descriptors.columns[feature_selector.get_support()] #restore the column names
new_bittersweet_descriptors = pd.DataFrame(filter_bittersweet_descriptors, columns=res_descriptors)
new_bittersweet_descriptors


,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_phos_acid,fr_piperdine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_unbrch_alkane,fr_urea
0,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,302.282,288.170,302.079038,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.931030,11.931030,0.069444,-1.054352,0.755242,27.444444,252.354,228.162,252.172545,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.709217,12.709217,0.094012,-1.412606,0.477325,19.000000,346.291,332.179,346.068867,130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.921813,12.921813,0.155771,-1.473063,0.465280,18.370370,376.317,360.189,376.079432,142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.888759,10.888759,0.057655,-1.223235,0.469461,25.045455,302.282,288.170,302.079038,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,13.298427,13.298427,0.040278,0.040278,0.716024,43.296296,364.445,340.253,364.178693,140.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2363,11.864139,11.864139,0.172040,-1.074904,0.585158,13.000000,308.334,288.174,308.137222,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2364,9.119537,9.119537,0.525787,-1.568935,0.290153,49.250000,180.156,168.060,180.063388,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2365,11.933569,11.933569,0.365963,-1.781427,0.236373,30.869565,332.261,316.133,332.074347,128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# print(chemtaste_bittersweet_df["Class taste"].value_counts())

# bittersweet_train_df, bittersweet_test_df = train_test_split(
#     chemtaste_bittersweet_df, 
#     test_size=0.2,          # 20% for testing
#     random_state=42,        # Ensures reproducibility
#     shuffle=True            # Shuffle data before splitting
# )
# print("Training set size:", len(bittersweet_train_df))
# print("Testing set size:", len(bittersweet_test_df))


Bitterness    1298
Sweetness     1069
Name: Class taste, dtype: int64
Training set size: 1893
Testing set size: 474


In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    new_bittersweet_descriptors, chemtaste_bittersweet_df['Class taste'], test_size=0.2
)

In [ ]:
# with open('missing.txt', 'w') as f:
#     f.write(X_train.isna().sum().to_string())

2367

In [42]:
rf_classifier = RandomForestClassifier(    
    n_estimators=250,      # Balance performance and speed
    # max_depth=10,          # Prevent overfitting
    # min_samples_split=5,   # Require more samples to split
    # min_samples_leaf=2,    # Reduce noise in leaves
    # max_features='sqrt',   # sqrt(148) ≈ 12 features per split
    # class_weight='balanced',  # If classes are imbalanced
    # random_state=42,       # Reproducibility
    # n_jobs=-1             # Use all CPU cores
)
# Fit the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=250)

In [43]:
pred = rf_classifier.predict(X_test)

In [44]:
from sklearn.metrics import classification_report

print(classification_report(pred, y_test, digits=3))

              precision    recall  f1-score   support

  Bitterness      0.931     0.910     0.921       268
   Sweetness      0.887     0.913     0.900       206

    accuracy                          0.911       474
   macro avg      0.909     0.912     0.910       474
weighted avg      0.912     0.911     0.912       474

